In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.375
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from geocat.comp import eofunc_eofs, eofunc_pcs
from datetime import datetime
import warnings
from matplotlib.patches import Rectangle
import pdo_functions
import importlib
importlib.reload(pdo_functions)
import random
import numba
import statsmodels.api as sm
import matplotlib.gridspec as gridspec

In [ ]:
# Open SST data and compute the PDO and Nino3.4
sst = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/HadISST_sst.nc')['sst']

pdo = pdo_functions.pdo_from_hadisst(sst, 1980, 2010)

nino_34 = pdo_functions.calc_nino_34_timeseries(sst, False, 1980, 2010)

In [ ]:
# land mask
land_110 = regionmask.defined_regions.natural_earth_v4_1_0.land_110
sst_land = xr.where(land_110.mask_3D(sst)==False, 1, np.nan).squeeze()

# Apply land mask 
sst_no_land = sst*sst_land

# Uniform sea ice temperature
sst_no_land_fill = sst_no_land.where(sst_no_land > -1.8, -1.8)

In [ ]:
pdo_pattern = pdo_functions.linregress_3D(pdo, sst_no_land_fill)[2]

In [ ]:
pdo_pattern.min()

In [ ]:
pdo[pos_pdo]

In [ ]:
fig = plt.figure(figsize=(5, 4.75))

colormap = cmr.fusion
blue_rgb = colormap(0.8)
red_rgb = colormap(0.2)
blue_hex = c.to_hex(blue_rgb)
red_hex = c.to_hex(red_rgb)

bounds = [-0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, -0.025, 0.025, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

norm = c.BoundaryNorm(bounds, plt.get_cmap('cmr.fusion_r').N)

gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])

pos_pdo = (pdo > 0)
neg_pdo = (pdo < 0)

ax1 = fig.add_subplot(gs[0], projection=ccrs.Robinson(central_longitude=180))
ax1.coastlines(lw=0.25, color='xkcd:gunmetal')
ax1.spines['geo'].set_edgecolor('xkcd:gunmetal')
ax1.spines['geo'].set_linewidth(0.25)
ax1.set_facecolor('xkcd:light gray')

ax1.pcolormesh(sst['longitude'], sst['latitude'], pdo_pattern*sst_land, transform=ccrs.PlateCarree(), norm=norm, cmap='cmr.fusion_r')

ax1.set_title('Pacific Decadal Oscillation', size=8, pad=3)

cax = plt.axes([0.1, .37, 0.8, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', ticks=[-0.7, -0.5, -0.3, -0.1, 0, 0.1, 0.3, 0.5, 0.7])
cbar.set_label(r'$^{\circ}$C', size=6, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=6, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.5)
cbar.outline.set_color('black')
# cbar.outline.set_visible(False)

ax2 = fig.add_subplot(gs[1])
ax2.bar(np.arange(1870, 2025, 1/12)[pos_pdo], pdo[pos_pdo], width=0.07, color=red_hex)
ax2.bar(np.arange(1870, 2025, 1/12)[neg_pdo], pdo[neg_pdo], width=0.07, color=blue_hex)
ax2.tick_params(labelsize=6, width=0.25, color='xkcd:gunmetal')

ax2.set_ylabel('standard deviations', size=6)
ax2.set_xlabel('time', size=6)

# ax2.set_title('time series', size=8, pad=3)

for spine in ax2.spines.values():
    spine.set_linewidth(0.25)
    spine.set_edgecolor('xkcd:gunmetal')